In [1]:
from datasets import load_dataset, Dataset
from torch.nn import LSTM
import torch
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


/home/brian/anaconda3/envs/cse447/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [79]:
ds_train = load_dataset("uonlp/CulturaX", "en", split='train', streaming=True)
ds_test = load_dataset("uonlp/CulturaX", "en", split='test', streaming=True)
ds_val = load_dataset("uonlp/CulturaX", "en", split='validation', streaming=True)

ValueError: Bad split: test. Available splits: ['train']

In [ ]:
ds_head = ds.take(150)
for d in ds_head:
    print(d)
    break

{'text': 'DOT Announces 2008 Exploration Program - Redorbit\nCALGARY, ALBERTA--(Marketwire - July 31, 2008) - DOT Resources Ltd. (TSX VENTURE:DOT) ("DOT" or the "Corporation") is pleased to announce it will be completing a deep 3D Induced Polarization/ Resistivity ("IP") survey on its Dot porphyry copper property (the "Property") located 17 kilometres south of the Highland Valley Mining complex, in central British Columbia. The purpose of the 3D IP survey is to test the depth extension of the copper-molybdenum mineralization outlined to date on the Southeast zone and to explore the area of low magnetic susceptibility for zones of sulphide mineralization.\nThe total field magnetometer survey completed over the Property in 2007 located a large area measuring 3 kilometres ("km") by 1 km of low magnetic susceptibility (see Figure 1) which is interpreted to reflect either:\n- alteration, or\n- the intersection of cross-cutting structures, or\n- a combination of both features\nThe porphyry d

In [29]:
def create_pairs(example):
    text = example["text"]
    pairs = []
    for i in range(1, len(text)):
        inputs = text[:i]
        outputs = text[i]
        if len(inputs) > 100:
            inputs = inputs[-100:]
        pairs.append({"input": inputs, "output": outputs})
    return pairs
data_pairs = []
for d in ds_head:
    data_pairs.extend(create_pairs(d))

In [30]:
chars = set()
num_chars = 0
for d in ds_head:
    num_chars += len(d['text'])
    chars.update(d['text'])
print(len(chars))
print(num_chars)

187
438048


In [31]:
char_to_idx = {char: idx + 1 for idx, char in enumerate(sorted(chars))}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

def encode_sequence(seq):
    return [char_to_idx[char] for char in seq]

In [32]:
print(char_to_idx)
print(idx_to_char)

{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '#': 6, '$': 7, '%': 8, '&': 9, "'": 10, '(': 11, ')': 12, '*': 13, '+': 14, ',': 15, '-': 16, '.': 17, '/': 18, '0': 19, '1': 20, '2': 21, '3': 22, '4': 23, '5': 24, '6': 25, '7': 26, '8': 27, '9': 28, ':': 29, ';': 30, '<': 31, '=': 32, '>': 33, '?': 34, '@': 35, 'A': 36, 'B': 37, 'C': 38, 'D': 39, 'E': 40, 'F': 41, 'G': 42, 'H': 43, 'I': 44, 'J': 45, 'K': 46, 'L': 47, 'M': 48, 'N': 49, 'O': 50, 'P': 51, 'Q': 52, 'R': 53, 'S': 54, 'T': 55, 'U': 56, 'V': 57, 'W': 58, 'X': 59, 'Y': 60, 'Z': 61, '[': 62, '\\': 63, ']': 64, '^': 65, '_': 66, '`': 67, 'a': 68, 'b': 69, 'c': 70, 'd': 71, 'e': 72, 'f': 73, 'g': 74, 'h': 75, 'i': 76, 'j': 77, 'k': 78, 'l': 79, 'm': 80, 'n': 81, 'o': 82, 'p': 83, 'q': 84, 'r': 85, 's': 86, 't': 87, 'u': 88, 'v': 89, 'w': 90, 'x': 91, 'y': 92, 'z': 93, '{': 94, '|': 95, '}': 96, '~': 97, '\x9d': 98, '£': 99, '§': 100, '©': 101, '«': 102, '®': 103, '·': 104, 'º': 105, '»': 106, 'à': 107, 'á': 108, 'â': 109, 'ã': 110, '

In [33]:
sequences = []
outputs = []
for d in data_pairs:
    sequences.append(torch.tensor(encode_sequence(d["input"]), dtype=torch.long))
    outputs.append(torch.tensor(char_to_idx[d["output"]], dtype=torch.long))
X_train = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=0)
y_train = torch.stack(outputs)


In [34]:
X_train = X_train.to(device)
y_train = y_train.to(device)

In [35]:
print(X_train.shape)
print(X_train[0])

torch.Size([437948, 100])
tensor([39,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0], device='cuda:0')


In [36]:
print(y_train.shape)

torch.Size([437948])


In [37]:
class CharDataset(Dataset):
    def __init__(self, sequences, outputs):
        self.sequences = sequences
        self.outputs = outputs

    def __len__(self):
        return self.sequences.shape[0]

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        output = self.outputs[idx]
        return sequence, output


In [38]:

train_dataset = CharDataset(X_train, y_train)
dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

torch.save({'X': X_train, 'y': y_train}, "./train_dataset.pt")

In [39]:
data = torch.load("./train_dataset.pt")
train_dataset = CharDataset(data['X'], data['y'])
dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [40]:
import torch.nn as nn

class CharLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers):
        super(CharLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)  # Output size is vocab_size

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # Take the last LSTM output
        return out

In [ ]:
import torch.optim as optim

# Model parameters
vocab_size = len(char_to_idx) + 1
embed_dim = 12
hidden_dim = 128
num_layers = 4

# Initialize model
model = CharLSTM(vocab_size, embed_dim, hidden_dim, num_layers)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
count = 0
epochs = 10
for epoch in range(epochs):
    for inputs, targets in dataloader:
        count += 1
        optimizer.zero_grad()
        outputs = model(inputs)  # (batch_size, vocab_size)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Loss: 3.5071
Loss: 3.5341
Loss: 3.1997
Loss: 3.4474
Loss: 3.2438
Loss: 3.1500
Loss: 3.3261
Loss: 2.9506
Loss: 4.1606
Loss: 3.2293
Loss: 3.1494
Loss: 3.1633
Loss: 3.4746
Loss: 3.1419
Loss: 3.1628
Loss: 3.3034
Loss: 2.7172
Loss: 3.3559
Loss: 3.3383
Loss: 3.2977
Loss: 3.2969
Loss: 3.0898
Loss: 3.1281
Loss: 3.2703
Loss: 3.1861
Loss: 3.1769
Loss: 2.9191
Loss: 3.0554
Loss: 3.4426
Loss: 3.3386
Loss: 3.1336
Loss: 3.4490
Loss: 3.2310
Loss: 3.3383
Loss: 3.2543
Loss: 2.9732
Loss: 3.0182
Loss: 3.1868
Loss: 3.1471
Loss: 3.0621
Loss: 3.4064
Loss: 3.2404
Loss: 2.9424
Loss: 3.1476
Loss: 3.4871
Loss: 2.9830
Loss: 3.5914
Loss: 3.4563
Loss: 3.1582
Loss: 3.0937
Loss: 3.2334
Loss: 3.5465
Loss: 3.3893
Loss: 3.4133
Loss: 3.6386
Loss: 3.4696
Loss: 3.4246
Loss: 3.2072
Loss: 3.1458
Loss: 3.1667
Loss: 2.8759
Loss: 2.9735
Loss: 2.9470
Loss: 3.2661
Loss: 3.2276
Loss: 3.3414
Loss: 2.8227
Loss: 2.9724
Loss: 3.3293
Loss: 3.1119
Loss: 2.8708
Loss: 3.1238
Loss: 2.5376
Loss: 2.9613
Loss: 3.1822
Loss: 3.4488
Loss: 3.0990

In [78]:
for inputs, targets in dataloader:
    outputs = model(inputs)
    print(outputs.shape)
    top3_preds = torch.topk(outputs, 3, dim=1)[1]
    translated_input = [idx_to_char[idx.item()] for idx in inputs[0]]
    translated_output = [idx_to_char[idx.item()] for idx in top3_preds[0]]
    translated_target = idx_to_char[targets[0].item()]

    print("Inputs:", ''.join(translated_input))
    print("Predicted:", translated_output)
    print("Targets:", translated_target)
    break

torch.Size([32, 188])
Inputs: nd have already executed a Waiver of Rights and endorsement for the lifting of the Notice of Acquisi
Predicted: ['t', 'c', 'n']
Targets: t
